In [94]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import seaborn as sns

In [78]:
train_path = "H:\\Hobby\\ML Dataset\\titanic\\train.csv"
test_path = "H:\\Hobby\\ML Dataset\\titanic\\test.csv"

In [79]:
train = pd.read_csv(train_path)
test = pd.read_csv(test_path)


y=train['Survived']
train=train.drop(['Survived','PassengerId', 'Ticket'], axis=1)
passID = test['PassengerId']
test = test.drop(['PassengerId', 'Ticket'], axis=1)

In [80]:
train = train.drop(['Cabin'], axis=1)
test = test.drop(['Cabin'], axis=1)

In [81]:
train_test_data = [train, test] 

for dataset in train_test_data:
    dataset['Title'] = dataset['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)

train = train.drop(['Name'], axis=1)
test = test.drop(['Name'], axis=1)

In [82]:
cat_col = []; num_col = []
for col in train.columns:
    if train[col].dtypes=='object':
        cat_col.append(col)
    else:
        num_col.append(col)

In [83]:
num_trans = SimpleImputer(strategy='constant')

In [84]:
cat_trans = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

In [85]:
preprocessor = ColumnTransformer(
transformers=[
    ('num', num_trans, num_col),
    ('cat', cat_trans, cat_col)
])

In [96]:
xTrain, xVal, yTrain, yVal = train_test_split(train, y)
#model = DecisionTreeClassifier(max_leaf_nodes=35, random_state=0)
model = RandomForestClassifier(n_estimators=300)

In [97]:
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

pipeline.fit(xTrain, yTrain)
preds = pipeline.predict(xVal)
score = accuracy_score(preds, yVal)

score

0.7713004484304933

In [98]:
for col in train.columns:
    print(train[col].isnull().sum(), col)

0 Pclass
0 Sex
177 Age
0 SibSp
0 Parch
0 Fare
2 Embarked
0 Title


In [99]:
pipeline.fit(train, y)
preds = pipeline.predict(test)

output = pd.DataFrame({'PassengerId': passID,
                      'Survived': preds})

output.to_csv('5thSubmission.csv', index=False)